Домашнее задание 2: библиотека Numpy и линейная алгебра

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import ndimage  # спецификатор для работы с изображениями
from scipy import misc
import scipy.stats as sps

from PIL import Image

from random import randint

%matplotlib inline

Что такое NumPy? - это питоновская библиотека для работы с массивами 
Зачем используется и почему работает быстрее? - сам по себе питон не адаптирован к простой работе с массивами, а она адаптирована 
Что такое линейная алгебра? - это раздел математики, в котором оперируют матрицами
Как она связана с машинным обучением и глубоким обучением? - в машинном обучении мы будем оперировать матрицами

Задание 1

В первой задаче вам предлагается перемножить две квадратные матрицы двумя способами -- без использования пакета numpy и с ним.

In [ ]:
# Для генерации матриц используем фукнцию random -- она используется для генерации случайных объектов 
# функция sample создает случайную выборку. В качестве аргумента ей передается кортеж (i,j), здесь i -- число строк,
# j -- число столбцов.
a = np.random.sample((10,10))
b = np.random.sample((10,10)) 
# выведите размерность (ранг) каждой матрицы с помощью функции ndim.
print(a.ndim)
print(b.ndim)
# Используйте функцию shape, что она вывела?
print(a.shape) # shape даёт число элементов по каждому из направлений матрицы
print(b.shape)
# ========
# тут можно писать код

def sc_matrix(matrix_first, matrix_second):
    ''' функция скалярного перемножения 2 матриц без np '''
    b = len(matrix_first[0]) != len(matrix_second)
    if b:
        return print('оу оу оу, такие матрицы перемножать скалярно не выйдет, число столбцов первой матрицы не равно числу строк второй')
    else:
        result_matrix = [[0 for el in range(0, len(matrix_second[0]))] for le in range(0, len(matrix_first))]
        for i in range(0, len(matrix_first)):
            for j in range(0, len(matrix_second[0])):
                for r in range(0, len(matrix_first[0])):
                    result_matrix[i][j] += matrix_first[i][r]*matrix_second[r][j]
        return np.array(result_matrix)
    
    pass

def sc_matrix_np(matrix_first, matrix_second):
    try:
        return np.dot(matrix_first, matrix_second)
    except ValueError as ve:
        return print(f'ошибка: {ve}')

# ========
print(f'{a}\n')
print(b)

In [ ]:
%%timeit
# засечем время работы функции без NumPy
sc_matrix(a,b)

In [ ]:
%%timeit
# засечем время работы функции с NumPy
sc_matrix_np(a,b)

Разница $10^3$ нумпай впереди


Задание 2

Напишите функцию, которая по данной последовательности $\{A_i\}_{i=1}^n$ строит последовательность $S_n$, где $S_k = \frac{A_1 + ... + A_k}{k}$. Аналогично -- с помощью библиотеки NumPy и без нее. Сравните скорость, объясните результат.

In [9]:
n = int(input('Enter n, число элементов последовательности : '))
#a_list = np.random.sample((n,1))
#print(a_list)
# без библиотеки нумпай
a_list_sec = [randint(0, 10) for i in range(0, n)] # sps.uniform.rvs(size=10 ** 7) #
A = a_list_sec
#print(a_list_sec)
#print(a_list_sec.ndim)
#print(a_list_sec.shape)
#print(a_list_sec.size)
def wo_np_list(a_list_sec):
    ''' функция считает последовательность без юзания нумпая '''
    # print(f'исходный вектор : {a_list_sec}')
    a_sum = 0
    s_list = [0 for i in range(0, len(a_list_sec))]
    for i, j in enumerate(a_list_sec):
        a_sum += j
        s_list[i] = round(a_sum / (i+1), 2)
    return s_list # f'конечная последовательность : {s_list}'

#s_list = wo_np_list(a_list_sec)
#print(s_list)

# с библиотекой нумпай
def w_np_list(a_list):
    ''' функция считает последовательность с юзанием нумпая '''
    a_list_sec = np.array(a_list)
    # print(f'исходный вектор : {a_list_sec}')
    a_list_test = np.array([round(a_list_sec[:i].sum()/(i),2) for i in range(1, n+1)])
    return a_list_test # f'конечная последовательность : {a_list_test}'

#s_list = w_np_list(a_list_sec)
#print(s_list)

Enter n, число элементов последовательности : 3


In [22]:
%timeit S1 = wo_np_list(A)
%timeit S2 = w_np_list(A)
S1 = wo_np_list(A)
S2 = w_np_list(A)
#проверим корректность:
np.abs(S1 - S2).sum()

3.53 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
31.5 µs ± 3.39 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


0.0

Задание 3

Пусть задан некоторый массив $X$. Надо построить новый массив, где все элементы с нечетными индексами требуется заменить на число $a$ (если оно не указано, то на 1). Все четные элементы исходного массива нужно возвести в куб и записать в обратном порядке относительно позиций этих элементов. Массив $X$ при этом должен остаться без изменений. В конце требуется слить массив $X$ с преобразованным $X$ и вывести в обратном порядке.

In [57]:
x_list = [randint(-10, 10) for i in range(0, randint(0, 50))]
def wo_np_list(x_list, a = 1):
    ''' без нумпая делаем всё что нужно '''
    xx_list_rev = x_list[::-1]
    xx_list = [a if i%2 == 0 else xx_list_rev[i]**3 for i in range(0, len(x_list))]
    return (x_list + xx_list)[::-1]
print(wo_np_list(x_list))

def wo_np_list_sec(x_list, a = 1):
    ''' без нумпая делаем всё что нужно. не через генератор '''
    xx_list_rev = x_list[::-1]
    xx_list = [a if i%2 == 0 else xx_list_rev[i]**3 for i in range(0, len(x_list))]
    return (x_list + xx_list)[::-1]

def w_np_list(x_list, a = 1):
    ''' с нумпаем делаем всё что нужно '''
    x_list_second = np.array(x_list)
    x_list = np.array(x_list)
    x_list_one[::4] = a / (x_list**3) 
    x_list_rev =  x_list[::-1] * 
    
    


[1, -729, 1, 125, 1, 64, 1, -8, 1, 512, 1, 216, 1, -216, 1, 0, -6, -1, 6, -3, 8, -5, -2, 3, 4, 0, 5, -6, -9, -6]


In [101]:
x_list = [2 for i in range(0, randint(0, 50))] #randint(1, 10)
x_list_second = np.array(x_list)
print(x_list_second)
x_list = np.array(x_list)
list_one = np.ones((len(x_list)), dtype=int)
i = 0
for i in range(x_list_second.ndim):
    if i 
    x_list_second[i]
print(x_list)
print(x_list.ndim)
#x_list_rev =  x_list[::-1]

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[8 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
1
